In [430]:
import pandas as pd
import requests
import re


## Overview

This workbook details an ETL (Extract, Transform, Load) process focused on analyzing traffic incident data, particularly homicides resulting from vehicular accidents. An essential part of this process involved translating all relevant dataset information into English, ensuring clarity and accessibility in analysis. The ETL process leveraged `high-quality datasets` and required `minimal transformation`, which primarily involved `date-time normalization, addressing missing values, and data type conversions`. Additionally, a notable effort was made to enrich the dataset with external weather information using the `Visual Crossing` API. However, this task faced challenges due to API key activation issues. Overall, the workbook lays the groundwork for a comprehensive analysis aimed at enhancing road safety and reducing traffic-related fatalities. It ensures that subsequent analyses are based on accurate, reliable, and well-understood data, crucial for informed decision-making and policy development.


## Data Extraction
- Converted the excel files provided in the started pack into csv and load them onto the workbook

In [431]:

file_path = '../data/raw/hechos_homicidios.csv'
df_homicides = pd.read_csv(file_path, encoding='ISO-8859-1')

df_homicides_clean = df_homicides.copy()

In [432]:
df_homicides_clean.head()

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Direcci¢n Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1.0,01/01/2016,2016.0,1.0,1.0,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8.0,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1.0,02/01/2016,2016.0,1.0,2.0,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9.0,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1.0,03/01/2016,2016.0,1.0,3.0,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1.0,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1.0,10/01/2016,2016.0,1.0,10.0,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8.0,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1.0,21/01/2016,2016.0,1.0,21.0,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PE¥A,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÇ?A, LUIS, PRES.",1.0,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


In [433]:
df_homicides_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10493 entries, 0 to 10492
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID                     696 non-null    object 
 1   N_VICTIMAS             696 non-null    float64
 2   FECHA                  696 non-null    object 
 3   AAAA                   696 non-null    float64
 4   MM                     696 non-null    float64
 5   DD                     696 non-null    float64
 6   HORA                   696 non-null    object 
 7   HH                     696 non-null    object 
 8   LUGAR_DEL_HECHO        696 non-null    object 
 9   TIPO_DE_CALLE          696 non-null    object 
 10  Calle                  695 non-null    object 
 11  Altura                 129 non-null    float64
 12  Cruce                  525 non-null    object 
 13  Direcci¢n Normalizada  688 non-null    object 
 14  COMUNA                 696 non-null    float64
 15  XY

In [434]:
df_homicides_clean.isna().sum()

ID                        9797
N_VICTIMAS                9797
FECHA                     9797
AAAA                      9797
MM                        9797
DD                        9797
HORA                      9797
HH                        9797
LUGAR_DEL_HECHO           9797
TIPO_DE_CALLE             9797
Calle                     9798
Altura                   10364
Cruce                     9968
Direcci¢n Normalizada     9805
COMUNA                    9797
XY (CABA)                 9797
pos x                     9797
pos y                     9797
PARTICIPANTES             9797
VICTIMA                   9797
ACUSADO                   9797
dtype: int64

## Data Transformation:
- Step 1: Removed redundant columns `AAAA`, `MM`, `DD`, `XY (CABA)`
- Step 2: Removed empty rows caused by conversion of excel to csv
- Step 3: Checked for duplicates and none where found
- Step 4: Corrected Data types and created a `datetime` column
- Step 5: Renaming all columns to correct language for smoothness in data analysis: English

**Note:** Leftover missing values were considered normal and droping these rows would have caused the loss of crucial data

**Remove Redundant Columns**

In [435]:
df_homicides_clean.drop(['AAAA', 'MM', 'DD'], axis=1, inplace=True)

In [436]:

df_homicides_clean.drop(['XY (CABA)'], axis=1, inplace=True)


**Remove Empty Rows**

In [437]:
df_homicides_clean.dropna(how='all', inplace=True)

In [438]:
# Check missing values and deem them usable 
missing_direccion_normalizada = df_homicides_clean[df_homicides_clean['Direcci¢n Normalizada'].isnull()]

missing_direccion_normalizada.head(30)


,ID,N_VICTIMAS,FECHA,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Altura,Cruce,Direcci¢n Normalizada,COMUNA,pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
38,2016-0052,1.0,20/04/2016,20:00:00,20,AUTOPISTA LUGONES PK 10000,AUTOPISTA,"LUGONES, LEOPOLDO AV.",NaN,NaN,NaN,13.0,.,.,MOTO-SD,MOTO,SD
106,2016-0136,1.0,25/10/2016,00:00:00,0,AU BUENOS AIRES - LA PLATA KM. 4,AUTOPISTA,AUTOPISTA BUENOS AIRES - LA PLATA,NaN,NaN,NaN,4.0,.,.,MOTO-CARGAS,MOTO,CARGAS
119,2016-0151,1.0,18/11/2016,20:35:00,20,SD,CALLE,NaN,NaN,NaN,NaN,0.0,.,.,PEATON-SD,PEATON,SD
180,2017-0050,2.0,28/04/2017,11:08:08,11,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,AUTOPISTA,AUTOPISTA PERITO MORENO,NaN,NaN,NaN,9.0,.,.,MOTO-CARGAS,MOTO,CARGAS
181,2017-0051,1.0,01/05/2017,03:47:47,3,AU DELLEPIANE 2400,AUTOPISTA,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,NaN,NaN,NaN,7.0,.,.,AUTO-AUTO,AUTO,AUTO
313,2018-0039,1.0,21/04/2018,22:15:00,22,AUTOPISTA LUGONES KM 4.7,AUTOPISTA,"LUGONES, LEOPOLDO AV.",NaN,NaN,NaN,14.0,.,.,PEATON-AUTO,PEATON,AUTO
546,2020-0026,1.0,17/05/2020,06:40:00,6,"LUGONES, LEOPOLDO AV. KM 6,1",AUTOPISTA,"LUGONES, LEOPOLDO AV.",NaN,NaN,NaN,14.0,.,.,MOTO-OBJETO FIJO,MOTO,OBJETO FIJO
621,2021-0023,1.0,01/03/2021,09:20:00,9,"AU BUENOS AIRES LA PLATA KM 4,5",AUTOPISTA,AUTOPISTA BUENOS AIRES - LA PLATA,NaN,NaN,NaN,4.0,.,.,MOTO-CARGAS,MOTO,CARGAS


**Check for duplicates**

In [439]:

duplicates = df_homicides_clean[df_homicides_clean.duplicated()]
duplicates.head()


,ID,N_VICTIMAS,FECHA,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Altura,Cruce,Direcci¢n Normalizada,COMUNA,pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO


**Data type manipulation:**
- Regular columns into their respective type
- Merging Date and Time into one column. Then rearanging the order

In [440]:
df_homicides_clean['N_VICTIMAS'] = df_homicides_clean['N_VICTIMAS'].astype(int)
df_homicides_clean['COMUNA'] = df_homicides_clean['COMUNA'].astype(int)
df_homicides_clean['Altura'] = pd.to_numeric(df_homicides_clean['Altura'], errors='coerce')

In [441]:
df_homicides_clean['FECHA'] = pd.to_datetime(df_homicides_clean['FECHA'], format='%d/%m/%Y')
df_homicides_clean['HORA'] = pd.to_datetime(df_homicides_clean['HORA'], format='%H:%M:%S', errors='coerce').dt.time
df_homicides_clean['FECHA_HORA'] = pd.to_datetime(df_homicides_clean['FECHA'].astype(str) + ' ' + df_homicides_clean['HORA'].astype(str), errors='coerce')
cols = df_homicides_clean.columns.tolist()
hh_index = cols.index('HH')
cols.insert(hh_index + 1, cols.pop(cols.index('FECHA_HORA')))  
df_homicides_clean = df_homicides_clean[cols]

**Correcting Language:**

In [454]:
# Column Names
df_homicides_clean.rename(columns={
    'N_VICTIMAS': 'NUMBER_OF_VICTIMS',
    'FECHA': 'DATE',
    'HORA': 'TIME',
    'HH': 'HOUR',
    'LUGAR_DEL_HECHO': 'PLACE_OF_EVENT',
    'TIPO_DE_CALLE': 'TYPE_OF_STREET',
    'Calle': 'STREET',
    'Altura': 'HEIGHT',
    'Cruce': 'CROSSING',
    'Direcci¢n Normalizada': 'NORMALIZED_ADDRESS',
    'COMUNA': 'COMMUNE',
    'pos x': 'POS_X',
    'pos y': 'POS_Y',
    'PARTICIPANTES': 'PARTICIPANTS',
    'VICTIMA': 'VICTIM',
    'ACUSADO': 'ACCUSED',
    'FECHA_HORA': 'DATE_TIME'
}, inplace=True)

In [456]:
# Participant column
translation_dict = {
    'PEATON': 'Pedestrian',
    'MOTO': 'Motorcycle',
    'AUTO': 'Car',
    'CARGAS': 'Cargo Vehicle',
    'BICICLETA': 'Bicycle',
    'PASAJEROS': 'Passengers',
    'MOVIL': 'Emergency Vehicle',
    'OTRO': 'Other Vehicles',
    'SD': 'No Data'
}

def translate_participants(participants):
    parts = participants.split('-')
    translated_parts = [translation_dict.get(part, part) for part in parts]
    return '-'.join(translated_parts)

df_homicides_clean['PARTICIPANTS'] = df_homicides_clean['PARTICIPANTS'].apply(translate_participants)

In [461]:
# participant and victim column content
translation_dict = {
    'PEATON': 'Pedestrian',
    'MOTO': 'Motorcycle',
    'AUTO': 'Car',
    'CARGAS': 'Cargo Vehicle',
    'BICICLETA': 'Bicycle',
    'PASAJEROS': 'Passengers',
    'MOVIL': 'Emergency Vehicle',
    'OBJETO FIJO': 'Fixed Object',
    'PEATON_MOTO': 'Pedestrian_Motorcycle',
    'SD': 'No Data',
    'MULTIPLE': 'Multiple',
    'OTRO': 'Other',
    'TREN': 'Train'
}


def translate_term(term):
    return translation_dict.get(term, term) 

df_homicides_clean['VICTIM'] = df_homicides_clean['VICTIM'].apply(translate_term)
df_homicides_clean['ACCUSED'] = df_homicides_clean['ACCUSED'].apply(translate_term)

## Weather API Implementation: `Visual Crossing`
- Step 1: Create Function to fetch weather data
- Step 2: Add new Columns `Temperature`, `Precipitation`, `Cloud Cover`, `Sunset`
- Step 3: Itterate Through Each row
- Step 4: Rearrange columns
- Step 5: Correct formating and data type of new columns

**New Columns to save data**

In [443]:
df_homicides_clean['TEMPERATURE'] = None
df_homicides_clean['PRECIPITATION'] = None
df_homicides_clean['CLOUD_COVER'] = None
df_homicides_clean['SUNSET'] = None

**Weather Function**

In [444]:
def fetch_weather_data(api_key, latitude, longitude, date):
    url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{latitude},{longitude}/{date}"
    params = {
        "key": api_key,
        "include": "obs",
        "unitGroup": "metric"  # Add this line to request data in metric units
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching data: {response.status_code}, {response.text}")
        return None


api_key = "VNJ6ZMXYEF336V9WAXA732GLK"  # Replace with your Visual Crossing Weather API Key



**Iterate over each row and save data**

In [445]:
for index, row in df_homicides_clean.iterrows():
    latitude = row['POS_Y']
    longitude = row['POS_X']
    date = row['DATE'].strftime('%Y-%m-%d')

    weather_data = fetch_weather_data(api_key, latitude, longitude, date)

    if weather_data and 'days' in weather_data:
        day_data = weather_data['days'][0]
        temperature = day_data.get('temp', None)
        precipitation_type = day_data.get('preciptype', None)
        cloud_cover = day_data.get('cloudcover', None)
        sunset = day_data.get('sunset', None)

        df_homicides_clean.at[index, 'TEMPERATURE'] = temperature
        df_homicides_clean.at[index, 'PRECIPITATION'] = precipitation_type
        df_homicides_clean.at[index, 'CLOUD_COVER'] = cloud_cover
        df_homicides_clean.at[index, 'SUNSET'] = sunset
    else:
        print(f"Missing or incomplete weather data for {date} at ({latitude}, {longitude})")


Error fetching data: 400, Bad API Request:Invalid location parameter value.
Missing or incomplete weather data for 2016-04-20 at (., .)
Error fetching data: 400, Bad API Request:Invalid location parameter value.
Missing or incomplete weather data for 2016-10-25 at (., .)
Error fetching data: 400, Bad API Request:Invalid location parameter value.
Missing or incomplete weather data for 2016-11-18 at (., .)
Error fetching data: 400, Bad API Request:Invalid location parameter value.
Missing or incomplete weather data for 2016-12-27 at (., .)
Error fetching data: 400, Bad API Request:Invalid location parameter value.
Missing or incomplete weather data for 2017-04-10 at (., .)
Error fetching data: 400, Bad API Request:Invalid location parameter value.
Missing or incomplete weather data for 2017-04-28 at (., .)
Error fetching data: 400, Bad API Request:Invalid location parameter value.
Missing or incomplete weather data for 2017-05-01 at (., .)
Error fetching data: 400, Bad API Request:Invali

**Rearange Columns**

In [446]:

columns = ['ID', 'NUMBER_OF_VICTIMS', 'DATE', 'TIME', 'HOUR', 'DATE_TIME', 'SUNSET', 'TEMPERATURE', 'PRECIPITATION', 'CLOUD_COVER', 
           'PLACE_OF_EVENT', 'TYPE_OF_STREET', 'STREET', 'HEIGHT', 'CROSSING', 'NORMALIZED_ADDRESS',
           'COMMUNE', 'POS_X', 'POS_Y', 'PARTICIPANTS', 'VICTIM', 'ACCUSED']
df_homicides_clean = df_homicides_clean[columns]


**Formating and data types**

In [447]:
def extract_text(s):
    return re.sub("[^a-zA-Z]", "", str(s))

df_homicides_clean['PRECIPITATION'] = df_homicides_clean['PRECIPITATION'].apply(extract_text)

df_homicides_clean['SUNSET'] = pd.to_datetime(df_homicides_clean['SUNSET'], format='%H:%M:%S', errors='coerce').dt.time
df_homicides_clean['TEMPERATURE'] = pd.to_numeric(df_homicides_clean['TEMPERATURE'], errors='coerce')
df_homicides_clean['CLOUD_COVER'] = pd.to_numeric(df_homicides_clean['CLOUD_COVER'], errors='coerce')


**Final check of dataset**

In [462]:
df_homicides_clean.tail()

,ID,NUMBER_OF_VICTIMS,DATE,TIME,HOUR,DATE_TIME,SUNSET,TEMPERATURE,PRECIPITATION,CLOUD_COVER,...,STREET,HEIGHT,CROSSING,NORMALIZED_ADDRESS,COMMUNE,POS_X,POS_Y,PARTICIPANTS,VICTIM,ACCUSED
691,2021-0093,1,2021-12-13,17:10:00,17,2021-12-13 17:10:00,20:01:42,24.1,None,85.2,...,RIESTRA AV.,NaN,MOM,RIESTRA AV. y MOM,7,-58.43353773,-34.64561636,Motorcycle-Car,Motorcycle,Car
692,2021-0094,1,2021-12-20,01:10:00,1,2021-12-20 01:10:00,20:06:03,25.3,None,33.8,...,"DELLEPIANE, LUIS, TTE. GRAL.",NaN,LACARRA AV.,"DELLEPIANE, LUIS, TTE. GRAL. y LACARRA AV.",9,-58.46739825,-34.65117757,Motorcycle-Car,Motorcycle,Car
693,2021-0095,1,2021-12-30,00:43:00,0,2021-12-30 00:43:00,20:09:44,30.0,None,46.8,...,GAONA AV.,NaN,TERRADA,GAONA AV. y TERRADA,11,-58.47293407,-34.61984745,Motorcycle-Cargo Vehicle,Motorcycle,Cargo Vehicle
694,2021-0096,1,2021-12-15,10:30:00,10,2021-12-15 10:30:00,20:03:11,22.4,rain,67.6,...,"PERON, EVA AV.",4071.0,NaN,"PERON, EVA AV. 4071",9,-58.47066794,-34.65021673,Car-Cargo Vehicle,Car,Cargo Vehicle
695,2021-0097,1,2021-11-18,06:10:00,6,2021-11-18 06:10:00,19:39:33,19.1,None,2.0,...,PADRE CARLOS MUJICA,709.0,NaN,PADRE CARLOS MUGICA 709,1,-58.37976155,-34.58679619,Bicycle-Car,Bicycle,Car


In [449]:
df_homicides_clean.isna().sum()

ID                      0
NUMBER_OF_VICTIMS       0
DATE                    0
TIME                    1
HOUR                    0
DATE_TIME               1
SUNSET                 12
TEMPERATURE            12
PRECIPITATION           0
CLOUD_COVER            12
PLACE_OF_EVENT          0
TYPE_OF_STREET          0
STREET                  1
HEIGHT                567
CROSSING              171
NORMALIZED_ADDRESS      8
COMMUNE                 0
POS_X                   0
POS_Y                   0
PARTICIPANTS            0
VICTIM                  0
ACCUSED                 0
dtype: int64

In [450]:
df_homicides_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 696 entries, 0 to 695
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   ID                  696 non-null    object        
 1   NUMBER_OF_VICTIMS   696 non-null    int32         
 2   DATE                696 non-null    datetime64[ns]
 3   TIME                695 non-null    object        
 4   HOUR                696 non-null    object        
 5   DATE_TIME           695 non-null    datetime64[ns]
 6   SUNSET              684 non-null    object        
 7   TEMPERATURE         684 non-null    float64       
 8   PRECIPITATION       696 non-null    object        
 9   CLOUD_COVER         684 non-null    float64       
 10  PLACE_OF_EVENT      696 non-null    object        
 11  TYPE_OF_STREET      696 non-null    object        
 12  STREET              695 non-null    object        
 13  HEIGHT              129 non-null    float64       
 14 

## Loading/Saving the Data
1. Saved dataframes: `df_homicides_clean`
2. Saved the data in`.csv` 
3. File Path: `'../data/processed/'`

In [463]:
csv_file_path = '../data/processed/homicides_with_weather.csv'

df_homicides_clean.to_csv(csv_file_path, index=False)
